In [1]:
import os
import h5py
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from transformers import TFViTModel, ViTFeatureExtractor
from PIL import Image
from io import BytesIO
from sklearn.metrics import roc_curve, auc

2024-07-18 13:45:18.307519: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-18 13:45:18.307624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-18 13:45:18.431097: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Load data

In [2]:
# Load data
# define data path
data_path = '/kaggle/input/isic-2024-challenge/'

path_train_hdf5 = os.path.join(data_path, 'train-image.hdf5')
path_test_hdf5 = os.path.join(data_path, 'test-image.hdf5')

path_train_meta = data_path + 'train-metadata.csv'
path_test_meta = data_path + 'test-metadata.csv'

In [3]:
# load data
train_hdf5 = h5py.File(path_train_hdf5, 'r')
test_hdf5 = h5py.File(path_test_hdf5, 'r')

In [4]:
# Load metadata
train_meta = pd.read_csv(path_train_meta)
test_meta = pd.read_csv(path_test_meta)

/tmp/ipykernel_23/1940620682.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_meta = pd.read_csv(path_train_meta)


In [5]:
# read in the isic ids and target values
train_isic_ids = train_meta['isic_id'].values
train_isic_ids = train_meta[train_meta['lesion_id'].notnull()]['isic_id'].values

test_isic_ids = test_meta['isic_id'].values

train_targets = train_meta[train_meta['lesion_id'].notnull()]['target'].values

In [6]:
len(train_targets)

22058

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split

# Splits de data in train en validation sets
total_size = len(train_targets)
indices = np.arange(total_size)

# Definieer de verhoudingen voor train en val
train_size = int(total_size * 0.8)  # Pas dit aan naar jouw gewenste verhouding
val_size = total_size - train_size

# Gebruik train_test_split om indices te splitsen
train_indices, val_indices = train_test_split(indices, test_size=val_size, train_size=train_size)


In [8]:
print(train_indices, val_indices)

[21484  4037  3613 ... 10680  6719  5185] [ 6825 18516 17227 ...  9210  9772  7295]


In [9]:
def preprocess_image(image):
    image = tf.image.resize(image, [224, 224])
    if image.shape[-1] == 1:  # Handle grayscale images
        image = tf.image.grayscale_to_rgb(image)
    print(image.shape[-1])
    image = image[..., ::-1]
    return image

In [10]:
class ISICDataset(tf.keras.utils.Sequence):
    def __init__(self, hdf5_file, isic_ids, targets=None, feature_extractor=None, batch_size=32):
        self.hdf5_file = hdf5_file
        self.isic_ids = isic_ids
        self.targets = targets
        self.feature_extractor = feature_extractor
        self.batch_size = batch_size
        
    def __len__(self):
        return len(self.isic_ids)
    
    def __getitem__(self, idx):
        isic_id = str(self.isic_ids[idx])
        image = Image.open(BytesIO(self.hdf5_file[isic_id][()]))
        image = image.convert("RGB")
        image = np.array(image)
        if self.feature_extractor:
            image = self.feature_extractor(images=image, return_tensors="tf").pixel_values[0]
        if self.targets is not None:
            target = self.targets[idx]
            return image, target
        else:
            return image
    
    def _load_image(self, isic_id):
        isic_id = str(isic_id)
        image = Image.open(BytesIO(self.hdf5_file[isic_id][()]))
        image = image.convert("RGB")
        image = np.array(image)
        if self.feature_extractor:
            image = self.feature_extractor(images=image, return_tensors="tf").pixel_values[0]
        return image

    def _generator(self):
        for idx in range(len(self.isic_ids)):
            yield self._load_image(self.isic_ids[idx]), self.targets[idx]

    def to_tf_dataset(self):
        tf_dataset = tf.data.Dataset.from_generator(self._generator, 
                                                    output_signature=(tf.TensorSpec(shape=(3, 224, 224), dtype=tf.float32),
                                                                      tf.TensorSpec(shape=(), dtype=tf.int64)))
        tf_dataset = tf_dataset.batch(self.batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
        return tf_dataset

In [11]:

model_name = 'google/vit-base-patch16-224'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)

train_dataset = ISICDataset(train_hdf5, train_isic_ids[train_indices], targets=train_targets[train_indices], feature_extractor=feature_extractor)
val_dataset = ISICDataset(train_hdf5, train_isic_ids[val_indices], targets=train_targets[val_indices], feature_extractor=feature_extractor)


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [12]:
# Maak TensorFlow datasets
tf_train_dataset = train_dataset.to_tf_dataset()
tf_val_dataset = val_dataset.to_tf_dataset()

In [13]:
vit_model = TFViTModel.from_pretrained(model_name)

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing TFViTModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTModel were not initialized from the PyTorch model and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()

if "GPU" not in device_name:
    print("GPU device not found")
    
print('Found GPU at: {}'.format(device_name))

print("GPU", "available (YESS!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")

Found GPU at: /device:GPU:0
GPU available (YESS!!!!)


In [15]:
class CustomViTModel(tf.keras.Model):
    def __init__(self, backbone, num_classes):
        super(CustomViTModel, self).__init__()
        self.backbone = backbone
        self.dense = layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        with tf.device('/GPU:0'):
            outputs = self.backbone(inputs).last_hidden_state[:, 0, :]
            outputs = self.dense(outputs)
            return outputs

In [16]:
num_classes = len(np.unique(train_targets))
custom_vit_model = CustomViTModel(vit_model, num_classes)

In [17]:
# Compileer het model met een gebruikelijke loss-functie
custom_vit_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                         loss='sparse_categorical_crossentropy',
                         metrics=['accuracy'])

In [18]:
history = custom_vit_model.fit(tf_train_dataset, epochs=10, validation_data=tf_val_dataset)

Epoch 1/10


I0000 00:00:1721310372.132019      69 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1721310372.170224      69 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


    552/Unknown 838s 1s/step - accuracy: 0.9787 - loss: 0.1095

W0000 00:00:1721311187.901450      66 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
W0000 00:00:1721311195.908685      66 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


552/552 ━━━━━━━━━━━━━━━━━━━━ 1044s 2s/step - accuracy: 0.9787 - loss: 0.1095 - val_accuracy: 0.9835 - val_loss: 0.0722
Epoch 2/10


W0000 00:00:1721311393.300120      67 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


552/552 ━━━━━━━━━━━━━━━━━━━━ 927s 2s/step - accuracy: 0.9823 - loss: 0.0744 - val_accuracy: 0.9839 - val_loss: 0.0690
Epoch 3/10
552/552 ━━━━━━━━━━━━━━━━━━━━ 927s 2s/step - accuracy: 0.9827 - loss: 0.0710 - val_accuracy: 0.9841 - val_loss: 0.0674
Epoch 4/10
552/552 ━━━━━━━━━━━━━━━━━━━━ 930s 2s/step - accuracy: 0.9826 - loss: 0.0691 - val_accuracy: 0.9841 - val_loss: 0.0663
Epoch 5/10
552/552 ━━━━━━━━━━━━━━━━━━━━ 940s 2s/step - accuracy: 0.9827 - loss: 0.0677 - val_accuracy: 0.9839 - val_loss: 0.0656
Epoch 6/10
552/552 ━━━━━━━━━━━━━━━━━━━━ 949s 2s/step - accuracy: 0.9829 - loss: 0.0666 - val_accuracy: 0.9832 - val_loss: 0.0651
Epoch 7/10
552/552 ━━━━━━━━━━━━━━━━━━━━ 934s 2s/step - accuracy: 0.9828 - loss: 0.0656 - val_accuracy: 0.9835 - val_loss: 0.0647
Epoch 8/10
552/552 ━━━━━━━━━━━━━━━━━━━━ 938s 2s/step - accuracy: 0.9829 - loss: 0.0648 - val_accuracy: 0.9835 - val_loss: 0.0644
Epoch 9/10
552/552 ━━━━━━━━━━━━━━━━━━━━ 934s 2s/step - accuracy: 0.9829 - loss: 0.0641 - val_accuracy: 0.983

In [19]:
# Genereer voorspellingen op de testset
test_images = []
test_labels = []
for batch in tf_val_dataset:
    images, labels = batch
    test_images.append(images)
    test_labels.append(labels)

test_images = tf.concat(test_images, axis=0)
test_labels = tf.concat(test_labels, axis=0)


In [20]:

# Verkrijg de voorspellingen van het model
predictions = custom_vit_model.predict(test_images)
predicted_probabilities = tf.reduce_max(predictions, axis=1)

# Binarize de test labels voor ROC berekening
test_labels_binary = (test_labels == 1).numpy().astype(int)


  2/138 ━━━━━━━━━━━━━━━━━━━━ 25s 184ms/step

W0000 00:00:1721319967.544507      69 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


138/138 ━━━━━━━━━━━━━━━━━━━━ 32s 207ms/step


W0000 00:00:1721319995.927787      67 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [21]:

# Bereken de ROC-curve en de pAUC
fpr, tpr, thresholds = roc_curve(test_labels_binary, predicted_probabilities)
partial_auc = auc(fpr, tpr)

print(f'Partial AUC: {partial_auc}')

Partial AUC: 0.12587805107539965


In [22]:

# class CustomViTModel(tf.keras.Model):
#     def __init__(self, backbone, num_classes):
#         super(CustomViTModel, self).__init__()
#         self.backbone = backbone
#         self.dense = layers.Dense(num_classes, activation='softmax')

#     def call(self, inputs):
#         outputs = self.backbone(inputs).last_hidden_state[:, 0, :]
#         outputs = self.dense(outputs)
#         return outputs

In [23]:
# # Definieer de EmbeddingClassifierModel class
# class EmbeddingClassifierModel:
#     """
#     Wrapper model that creates a classification model out of an embedding
#     model and a reference dataset.
#     """

#     def __init__(self,
#                  model: CustomViTModel,
#                  reference_embeddings: np.ndarray,
#                  reference_labels: np.ndarray,
#                  distance_metric: str = 'euclidean'):
#         """
#         Initialize the model.
#         """
#         self.model = model
#         self.reference_embeddings = reference_embeddings
#         self.reference_labels = reference_labels
#         self.distance_metric = distance_metric
#         self.labels = np.unique(reference_labels)

#     def predict(self, image: Image) -> Mapping[str, float]:
#         image_embedding = self.model.predict(image[np.newaxis, ...])
#         return self._get_classification_scores(image_embedding)

#     def _get_classification_scores(self, image_embedding: np.ndarray) -> Mapping[str, float]:
#         """
#         Calculate the classification scores for one image embedding.
#         """
#         distances = cdist(image_embedding, self.reference_embeddings, metric=self.distance_metric)
#         classification = {}
#         for label in self.labels:
#             label_distances = distances[:, self.reference_labels == label]
#             classification[label] = np.mean(label_distances)
#         return classification


In [24]:

# # Gebruik de EmbeddingClassifierModel
# reference_embeddings = train_embeddings.numpy()  # Use the generated embeddings
# reference_labels = train_targets[train_indices]  # Use the training labels

# classifier = EmbeddingClassifierModel(custom_vit_model, reference_embeddings, reference_labels)

# # Voorbeeld classificatie
# test_image_id = test_isic_ids[0]
# test_image = Image.open(BytesIO(test_hdf5[test_image_id][()])).convert("RGB")
# test_image = feature_extractor(images=np.array(test_image), return_tensors="tf").pixel_values[0]

# classification_scores = classifier.predict(test_image)
# print("Classification Scores:", classification_scores)